In [1]:
import os

import pandas as pd

In [2]:
os.chdir("../")

In [3]:
%pwd

'C:\\Users\\kisho\\MyWork\\RedWineQuality'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [5]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH, 
                 schema_filepath = SCHEMA_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            l1_ratio=params.l1_ratio,
            alpha=params.alpha,
            target_column=schema.name
        )
        return model_trainer_config


In [10]:
import pandas as pd
from sklearn.linear_model import ElasticNet
import joblib

class ModelTrainer:
    
    def __init__(self, config: ModelTrainerConfig):
       self.config = config
    
    def train(self):
        
        train = pd.read_csv(self.config.train_data_path)
        test = pd.read_csv(self.config.test_data_path)
            
        X_train = train.drop(columns = [self.config.target_column])
        X_test = test.drop(columns = [self.config.target_column])
        y_train = train[self.config.target_column]
        y_test = test[self.config.target_column]
        
        print(X_train.shape, y_train.shape)
        
        en = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        en.fit(X_train, y_train)
        
        joblib.dump(en, os.path.join(self.config.root_dir, self.config.model_name))



In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-03-21 16:29:24,612: INFO: common: config\config.yaml file loaded successfully]
[2025-03-21 16:29:24,627: INFO: common: schema.yaml file loaded successfully]
[2025-03-21 16:29:24,627: INFO: common: params.yaml file loaded successfully]
Directory 'artifacts' already exists.
Directory 'artifacts/model_trainer' already exists.
(1279, 11) (1279,)
